In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import torch

import spacy
from spacy.tokens import Span
from edsnlp.pipelines.misc.external_model import ModelWrapper

from typing import List, Dict, Any

/export/home/tpetitjean/ecciMLConfit/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Defining a model

This model can be anything (PyTorch, SKLearn, etc.).  
Let's make an extremely basic sentiment analysis model.  

You can use GPU if you want here.

In [3]:
class SentimentAnalysisModel:
    def __init__(self):
        pass

    def batching(
        self,
        data: Dict[str, List[Any]],
        batch_size: int,
    ):
        i = 0
        n = len(data["text"])
        while i < n:
            yield {k: v[i : i + batch_size] for k, v in data.items()}
            i += batch_size

    def forward(
        self,
        batch,
    ):
        return [
            {
                "good": "good" in txt,
                "bad": "bad" in txt,
                "neutral": "neutral" in txt,
                "device": 'cuda' if torch.cuda.is_available() else "cpu"
            }
            for txt in batch["text"]
        ]
    
    def predict(
        self,
        data: Dict[str, List[Any]],
        batch_size: int,
    ):
        preds = []
        for batch in self.batching(data, batch_size):
            preds += self.forward(batch)
        return preds


We will test our model with some toy data

In [87]:
good_texts = [
    "It was a very good movie !",
    "The cheese was pretty good.",
    "It's gonna be a very good year, very good!",
]

bad_texts = [
    "I have a bad feeling about this",
    "This was pretty bad."
]

neutral = [
    "This is a neutral statement."
]

texts = 50*good_texts + 40*bad_texts + 30*neutral

random.shuffle(texts)

data = dict(text = texts)

In [88]:
model = SentimentAnalysisModel()
preds_from_model = model.predict(data, batch_size = 16)

In [89]:
preds_from_model[0], data["text"][0]

({'good': True, 'bad': False, 'neutral': False, 'device': 'cpu'},
 'The cheese was pretty good.')

## 2. Wrap the model

To use this model with EDS-NLP, you should wrap in by using the dedicated `ModelWrapper` class.  
Two parameters are available here:
- `span_getter`: To tell the wrapper how to generate inference data for your Model starting from a spaCy document
- `annotation_setter`: From the output predictions of your model, how do you set them on the starting spaCy Doc, Span or Token.

When creating your wrapper that inherits from `ModelWrapper`, you can either
- Use a pre-registered function for those two parameters
- Use your own by re-defining `self.span_getter` or `self.annotation_setter`

In [90]:
DEFAULT_SPAN_GETTER = {
    "@span_getters": "sentences",
}

DEFAULT_ANNOTATION_SETTER = {
    "@annotation_setters": "from-mapping",
    "mapping": {
        "good": "_.good",
        "bad": "_.bad",
        "neutral": "_.neutral",
        "device": "_.device",
    }
}

class SentimentAnalysisWrapper(ModelWrapper):
    def __init__(
        self,
        model: SentimentAnalysisModel,
        span_getter = DEFAULT_SPAN_GETTER,
        annotation_setter = DEFAULT_ANNOTATION_SETTER,
    ):
        super().__init__(model, span_getter, annotation_setter)
        
    
    def set_extensions(self):
        for ext in ["good", "bad", "neutral", "device"]:
            if not Span.has_extension(ext):
                Span.set_extension(ext, default=None)

In [91]:
wrap = SentimentAnalysisWrapper(model)

Finally we will save this wrapper model:

In [93]:
wrap.to_disk("./model.dill")

## 3. Use the wrapped model in a pipe

Use the `eds.external-model` pipe and give the pickled model path in the configuration

In [94]:
nlp = spacy.blank("eds")

nlp.add_pipe("eds.sentences")
nlp.add_pipe("eds.external-model", config=dict(model="./model.dill"))

Now simply use `nlp.pipe`:

In [96]:
spacy_preds = list(nlp.pipe(texts))

## 4. Sanity check

Let us check that the ouput if the model and the output of `nlp.pipe` matches up:

In [97]:
spacy_preds = [
    dict(
        good = doc[0].sent._.good,
        bad = doc[0].sent._.bad,
        neutral = doc[0].sent._.neutral,
        device = doc[0].sent._.device,
    ) for doc in spacy_preds
]

In [98]:
assert preds_from_model == spacy_preds